In [ ]:
import os
import shutil

# Root directory containing files
# Folder level 1
source_folder = '../../classedExeImg'

# Subfolders to divide
# Folder level 1
subfolder=['client1','client2','client3','client4','client5']

# Knowing that the subfolder structure is the same as the root folder structure at levels 2 and 3
# Folder level 2
type=['train','test','valid']
# Folder level 3
folder=['Locker', 'Mediyes', 'Winwebsec', 'Zbot', 'Zeroaccess']
folder2=['Locker', 'Mediyes', 'Winwebsec', 'Zbot', 'Zeroaccess','benign']

## Merge 2 folder 

In [ ]:
combine_folder = ['train', 'test']
# Create a temp folder to store the dataset
temp_folder = '../../classedExeImg/Temp'
if not os.path.exists(temp_folder):
    os.makedirs(temp_folder)
# Combine the train and test datasets into one temp folder
for dataset in combine_folder:
    for malware in folder:
        source_folder_path = source_folder + '/' + dataset + '/' + malware
        destination_folders = temp_folder + '/' + malware
        if not os.path.exists(destination_folders):
            os.makedirs(destination_folders)
        file_list = os.listdir(source_folder_path)
        for file in file_list:
            source_path = os.path.join(source_folder_path, file)
            dest_path = os.path.join(destination_folders, file)
            shutil.copy(source_path, dest_path)

In [ ]:
# benign_folder = '../../classedExeImg/Temp/benign'
# folder_move='../../classedExeImg/valid/benign'

# benign_file_list = os.listdir(benign_folder)
# for i in range(100):
#     source_path = os.path.join(benign_folder, benign_file_list[i])
#     dest_path = os.path.join(folder_move, benign_file_list[i])
#     shutil.move(source_path, dest_path)
#     print('Moved file:', benign_file_list[i])

### Take the merged folder and divide it into 5 smaller folders with a train/test ratio of 9/1

In [ ]:
temp_folder = '../../classedExeImg/Temp'
for malware in folder:
    source_folder_path = temp_folder+'/'+malware
    file_list = os.listdir(source_folder_path)
    # Split the file_list into train/test with a 9/1 ratio
    train_length = int(len(file_list) * 0.9)
    train_list = file_list[:train_length]
    test_list = file_list[train_length:]
    # Get the number of train and test samples for each client
    train_per_folder = len(train_list) // len(subfolder)
    test_per_folder = len(test_list) // len(subfolder)
    for client in subfolder:
        # Create train and test directories for each client
        des_folder="../../ClientData/"
        train_folder = des_folder+client+'/train/'+malware
        test_folder = des_folder+client+'/test/'+malware
        if not os.path.exists(train_folder):
            os.makedirs(train_folder)
        if not os.path.exists(test_folder):
            os.makedirs(test_folder)
        # Copy train and test datasets into the client's directory
        for i in range(train_per_folder):
            file=train_list.pop(0)
            source_path = os.path.join(source_folder_path, file)
            dest_path = os.path.join(train_folder, file)
            shutil.copy(source_path, dest_path)
        for i in range(test_per_folder):
            file=test_list.pop(0)
            source_path = os.path.join(source_folder_path, file)
            dest_path = os.path.join(test_folder, file)
            shutil.copy(source_path, dest_path)

## Split non-IID data, 1 client has n classes

In [ ]:
temp_folder = '../../classedExeImg/Temp'
target_folder = '../../NonIIDData'
#1/ Create an array containing the number of elements corresponding to n classes in a client
#For example, if len(arr)=2, it will divide into 2 classes for 1 client

#2/ Based on len(arr), we'll take the number of images in one class of temp_folder
#For example, if len(arr)=n, one client will have a maximum of n classes and
#the number of images in one class of the client will be len(number of images in one class of temp_folder)/n

#3/ Each element in array arr is an array containing corresponding classes for a client folder
#For example, if client 1 has len(arr[0])=2, then client 1 will have 2 classes corresponding to
#arr[0][0] and arr[1][0], client 2 will have 2 classes corresponding to arr[0][1] and arr[1][1]...
#and the value inside the array is the index of the class in the folder
#0 is Locker, 1 is Mediyes, 2 is Winwebsec, 3 is Zbot, 4 is Zeroaccess
arr=[
    [0,1,2,3,4],
    [4,0,1,2,3],
]
#First, create subfolder directories for clients in target_folder
#without train, test, valid but only with classes
#and the number of classes in each client will be len(arr)
for client in subfolder:
    for i in range(len(arr)):
        client_folder = target_folder+'/'+client+'/'+folder[arr[i][subfolder.index(client)]]
        if not os.path.exists(client_folder):
            os.makedirs(client_folder)
            
#Then distribute images to clients
for malware in folder:
    source_folder_path = temp_folder+'/'+malware
    file_list = os.listdir(source_folder_path)
    #Calculate the number of images to distribute to each client
    client_length = len(file_list) // len(arr)
    for client in subfolder:
        #Get the names of subfolders in the client's directory
        client_folder = target_folder+'/'+client
        list_folder = os.listdir(client_folder)
        #Check if the client contains that class folder
        if malware in list_folder:
            print(list_folder)
            #Distribute images to the client
            for j in range(client_length):
                file=file_list.pop(0)
                source_path = os.path.join(source_folder_path, file)
                dest_path = os.path.join(client_folder+'/'+malware, file)
                shutil.copy(source_path, dest_path)
            continue


## Split data for detection

In [24]:
def copy_files(temp_folder, client_folder,type, label, file):
    source_path = os.path.join(temp_folder, file)
    dir_path = os.path.join(client_folder+type+label)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    dest_path = os.path.join(client_folder+type+label, file)
    shutil.copy(source_path, dest_path)

In [ ]:
#Get all images from the Temp folder containing 5 malware classes
#Put them in the Malware folder (only take images, not folders)
temp_folder = '../../classedExeImg/Malware'
benign='../../classedExeImg/Temp/benign'

#Get all image files from temp folder and sort by size
#then divide into 3 parts, each part corresponding to 1 client
#then take these 3 parts, each part will be divided in ratio 9/1
#with 9 parts as train and 1 part as test (divided into train and test folders)
#Then do the same with the benign folder
#Finally, there will be a NonIIDDataDetect folder with 5 clients
# 2 folders client4, client5 only contain 1 benign class
# 2 folders client1, client2 only contain images from 5 malware classes
# 1 folder client3 contains both 5 malware classes and 1 benign class
#With ratio 9/1

target_folder = '../../NonIIDDataDetect'
#Create 5 client folders
for client in subfolder:
    client_folder = target_folder+'/'+client
    if not os.path.exists(client_folder):
        os.makedirs(client_folder)

list_file = os.listdir(temp_folder)
#Sort by size
list_file = sorted(list_file, key=lambda x: os.path.getsize(os.path.join(temp_folder, x)))
#Sort by size and divide into 2 parts
#keep the first part, sort the second part by name
list_file_1 = list_file[int(len(list_file)/2):]
list_file_2 = sorted(list_file[:int(len(list_file)/2)])
list_file = list_file_1 + list_file_2
print(len(list_file))
#Divide into 3 parts
part_length = len(list_file) // 3
part1 = list_file[:part_length]
part2 = list_file[part_length:2*part_length]
part3 = list_file[2*part_length:]
#Divide into 2 parts: train and test
train_length = int(part_length * 0.9)
train1 = part1[:train_length]
test1 = part1[train_length:]
train2 = part2[:train_length]
test2 = part2[train_length:]
train3 = part3[:train_length]
test3 = part3[train_length:]
#Divide benign into 3 parts
benign_list = os.listdir(benign)
benign_length = len(benign_list) // 3
benign1 = benign_list[:benign_length]
benign2 = benign_list[benign_length:2*benign_length]
benign3 = benign_list[2*benign_length:]
#Divide into 2 parts: train and test
train_benign_length = int(benign_length * 0.9)
train_benign1 = benign1[:train_benign_length]
test_benign1 = benign1[train_benign_length:]
train_benign2 = benign2[:train_benign_length]
test_benign2 = benign2[train_benign_length:]
train_benign3 = benign3[:train_benign_length]
test_benign3 = benign3[train_benign_length:]
label=['/malware','/benign']
#Put 3 parts of malware into 3 clients: client1, client2, client3
for client in subfolder:
    client_folder = target_folder+'/'+client
    if not os.path.exists(client_folder+'/train'):
        os.makedirs(client_folder+'/train')
    if not os.path.exists(client_folder+'/test'):
        os.makedirs(client_folder+'/test')
    if client=='client1':
        for file in train1:
            copy_files(temp_folder, client_folder, '/train', label[0], file)
        for file in test1:
            copy_files(temp_folder, client_folder, '/test', label[0], file)
    if client=='client2':
        for file in train2:
            copy_files(temp_folder, client_folder, '/train', label[0], file)
        for file in test2:
            copy_files(temp_folder, client_folder, '/test', label[0], file)
    if client=='client3':
        for file in train3:
            copy_files(temp_folder, client_folder, '/train', label[0], file)
        for file in test3:
            copy_files(temp_folder, client_folder, '/test', label[0], file)
        for file in train_benign1:
            copy_files(benign, client_folder, '/train', label[1], file)
        for file in test_benign1:
            copy_files(benign, client_folder, '/test', label[1], file)
    if client=='client4':
        for file in train_benign2:
            copy_files(benign, client_folder, '/train', label[1], file)
        for file in test_benign2:
            copy_files(benign, client_folder, '/test', label[1], file)
    if client=='client5':
        for file in train_benign3:
            copy_files(benign, client_folder, '/train', label[1], file)
        for file in test_benign3:
            copy_files(benign, client_folder, '/test', label[1], file)

8074
